Найти левую и правую полосы, определить на какой полоссе находится камера, найти точку схода указанных параллельных прямых(разделительная полоса и правая)

In [1]:
! pip install opencv-python


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
from scipy.cluster.vq import kmeans, vq

In [ ]:
def intersection_point(x1, y1, x2, y2, x3, y3, x4, y4): # координаты пересечения 2х прямых
    m1 = (y2 - y1) / (x2 - x1) if (x2 - x1) != 0 else float('inf')
    m2 = (y4 - y3) / (x4 - x3) if (x4 - x3) != 0 else float('inf')

    b1 = y1 - m1 * x1 if m1 != float('inf') else x1
    b2 = y3 - m2 * x3 if m2 != float('inf') else x3

    if m1 == m2:
        return None
    
    if m1 != float('inf') and m2 != float('inf'):
        x = (b2 - b1) / (m1 - m2)
        y = m1 * x + b1
    elif m1 == float('inf'):
        x = b1
        y = m2 * x + b2
    else:
        x = b2
        y = m1 * x + b1

    return (x, y)

In [ ]:
def cluster_lines(lines, k): # кластеризация на k кластеров
    if lines is None or len(lines) < k:
        return lines

    data=[]
    for line in lines:
        r, theta = line[0]
        data.append([r, theta])

    data = np.array(data, dtype=np.float32)
    centroids, _ = kmeans(data, k)

    labels, _ = vq(data, centroids)

    clustered_lines = []
    for i in range(k):
        cluster_points = data[labels == i]
        avg_r, avg_theta = np.mean(cluster_points, axis=0)
        clustered_lines.append([avg_r, avg_theta])

    return clustered_lines

Определим центральную и правую полосы

In [ ]:
image0 = cv2.imread('./images/input.png')
image = image0.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,550,650,apertureSize = 3)
lines = cv2.HoughLines(edges,1,np.pi/180,200)
final_lines = cluster_lines(lines, k=2)

lines_xy=[]
for line in final_lines:
    rho, theta = line
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    lines_xy.append([x1, y1, x2, y2])
    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
# cv2.imshow("Hough Lines", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Найдем и отметим точку схода

In [8]:
if len(final_lines) == 2:
    rho0, theta0 = final_lines[0]
    x1, y1, x2, y2 = lines_xy[0]
    x3, y3, x4, y4 = lines_xy[1]
    x, y = intersection_point(x1, y1, x2, y2, x3, y3, x4, y4)
    cv2.circle(image, (int(x), int(y)), 10, (0, 191, 255), -1)
cv2.imwrite('./images/output.png', image)

True

Найдем левую, центральную и правую полосы, определим где находится камера

In [24]:
image1 = image0.copy()
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
edges1 = cv2.Canny(gray1,400,650,apertureSize = 3)
# edges_colored1 = cv2.cvtColor(edges1, cv2.COLOR_GRAY2BGR)  # Перевод в 3 канала
# image1[edges > 0] = (0, 0, 255)
lines1 = cv2.HoughLines(edges1,1,np.pi/180,200)
final_lines1 = cluster_lines(lines1, k=3)
lines_xy1=[]
for line in final_lines1:
    rho, theta = line
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    lines_xy1.append([x1, y1, x2, y2])
    cv2.line(image1, (x1, y1), (x2, y2), (0, 255, 0), 2)

line_x_positions = []
height, width, _ = image0.shape
x_center_image = width // 2

for line in lines_xy1:
    x1, y1, x2, y2 = line
    x, _ = intersection_point(x1, y1, x2, y2, 0, height, x_center_image, height)
    line_x_positions.append((x, line))

line_x_positions.sort()
x = line_x_positions[1][0]
if (x <= x_center_image):
    print("The camera is in the right lane")
else:
    print("The camera is in the left lane")


The camera is in the right lane
